In [ ]:
using CSV, CurveFit
using NativeFileDialog: pick_folder
using Glob: glob
using Dates, DataFrames, Distributions
using LaTeXStrings
using Printf, Polynomials
using Statistics #, StatsPlotss
using StatsBase

In [ ]:
using DataFrames: DataFrame
using Dates: DateTime
using DSP: welch_pgram, freq, power, hanning
using Glob: glob
using NativeFileDialog: pick_folder

import DataFrames: Not, select!

# Widen screen for better viewing
display(HTML("<style>.jp-Cell { width: 120% !important; }</style>"))

# Function to calculate f2 and Pden2 using Welch's method
function calculate_spectra(heave_row, sample_frequency)
#######################################################
    
    ps_w = welch_pgram(heave_row, 256, 128; onesided=true, nfft=256, fs=sample_frequency, window=hanning)
    f2 = freq(ps_w)
    Pden2 = power(ps_w)
    
    return(f2, Pden2)

end    # calculate_spectra()


# Helper function to convert data without creating temporary strings
function parse_hex(data_array, idx)
###################################
    
    UInt16(data_array[idx]) << 8 | UInt16(data_array[idx+1])
    
end    # parse_hex()


# Function to check if the LSB is 1 (GPS interference)
function check_gps_flag(north_row)
##################################
    
    gps_flag_row = [((n & 0x1) == 1) ? 1 : 0 for n in north_row]
    
    return(gps_flag_row)
    
end    # check_gps_flag()
    

#######################################################################################################
#######################################################################################################
#######################################################################################################

# Define the path to the directory you want to search in
directory_path = pick_folder()

# Use glob to find all .RDT files in the directory and subdirectories
rdt_files = glob(".//*.RDT", directory_path)
rdt_files = rdt_files[1:end-1]  # Remove the file named TMP.RDT

# Initialize lists for date and hex matrices
utc = DateTime[]
heave_hex_matrix = Vector{Vector{UInt16}}()
north_hex_matrix = Vector{Vector{UInt16}}()
west_hex_matrix = Vector{Vector{UInt16}}()

@time begin
    
    for infil ∈ rdt_files
        
        println("Reading BINARY data from ", infil)
        data_array = reinterpret(UInt8, read(infil))

        ii = 1
        while ii < length(data_array)
            
            # Extract message header
            message_length = (UInt16(data_array[ii+2]) << 8) | UInt16(data_array[ii+3])

            # Extract timestamp
            yr = (UInt16(data_array[ii+5]) << 8) | UInt16(data_array[ii+6])
            month = data_array[ii+7]
            day = data_array[ii+8]
            hour = data_array[ii+9]
            minute = data_array[ii+10]

            push!(utc, DateTime(yr, month, day, hour, minute))

            # Sample frequency validation
            sample_rate_hex = UInt32(data_array[ii+11]) << 24 | UInt32(data_array[ii+12]) << 16 | 
                    UInt32(data_array[ii+13]) << 8 | UInt32(data_array[ii+14])
            sample_frequency = reinterpret(Float32, sample_rate_hex)

            if sample_frequency != 1.28f0
                error("Error: Sample rate not 1.28 Hz - Program terminated!")
            end

            rows = (message_length - 10) ÷ 6  # Calculate number of rows (samples) for this timestamp

            heave_row = Vector{UInt16}(undef, rows)
            north_row = Vector{UInt16}(undef, rows)
            west_row = Vector{UInt16}(undef, rows)

            @inbounds for jj ∈ 1:rows
                base_idx = ii + 15 + (jj - 1) * 6
                heave_row[jj] = parse_hex(data_array, base_idx)
                north_row[jj] = parse_hex(data_array, base_idx + 2)
                west_row[jj] = parse_hex(data_array, base_idx + 4)
            end

            push!(heave_hex_matrix, heave_row)
            push!(north_hex_matrix, north_row)
            push!(west_hex_matrix, west_row)

            ii += message_length + 6
            
        end
        
    end

    # Extract the Heave, North, and West values from the Hex data
    RDT_df = DataFrame(
        Date = utc, 
        Heave_hex = heave_hex_matrix, 
        North_hex = north_hex_matrix, 
        West_hex = west_hex_matrix
    )

    # Sort the DataFrame by date
    sort!(RDT_df, :Date)

    RDT_df.Heave = [reinterpret.(Int16, heave_row) ./ 100 for heave_row in RDT_df.Heave_hex]
    RDT_df.North = [reinterpret.(Int16, north_row) ./ 100 for north_row in RDT_df.North_hex]
    RDT_df.West = [reinterpret.(Int16, west_row) ./ 100 for west_row in RDT_df.West_hex]
    
    # Apply the function to each row in the North_hex column
    RDT_df.GPS_flag = [check_gps_flag(north_row) for north_row in RDT_df.North_hex]
    
    # Count the number of errors (sum of 1's) in each row of the GPS_Flag column
    RDT_df.GPS_errors = [sum(gps_flags) for gps_flags in RDT_df.GPS_flag]
    
    # Initialize arrays for f2 and Pden2
    f2_list = []
    Pden2_list = []

    # set sample frequency (in Hertz) for DWR-G and MkIII Waveriders
    sample_frequency = 1.28f0
    
    # Calculat spectra to each row in RDT_df.Heave using Welch's method
    for heave_row in RDT_df.Heave
        f2, Pden2 = calculate_spectra(heave_row, sample_frequency)
        push!(f2_list, f2)
        push!(Pden2_list, Pden2)
    end
    
    # Assign spectral data to new columns in RDT_df
    RDT_df.f2 = f2_list
    RDT_df.Pden2 = Pden2_list
    
    # Drop the Hex columns from RDT_df
    select!(RDT_df, !([:Heave_hex, :North_hex, :West_hex]))

end

### Locate all rows containing GPS errors

In [ ]:
bad = findall(row -> row.GPS_errors > 0, eachrow(RDT_df))

In [ ]:
using Plots
using Dates: Microsecond

using Dates: Dates
using CurveFit: curve_fit

# Function to apply polynomial fit to WSE's affected by GPS errors
# Uses selectable offset value to fine-tune result
function fix_gps_errors(heave_bad, date, gps_flag) 
##################################################
    
    heave = copy(heave_bad)
    
    gps_errors = findall(x -> x == 1, gps_flag)
    heave_length = length(heave)
    
    if !isempty(gps_errors)
        
        println(length(gps_errors), " GPS errors at ", Dates.format.(date, "yyyy-mm-dd HH:MM"))
        flush(stdout)
        
        for ii in reverse(gps_errors)

            error_center = ii

            if error_center <= 3
                error_center = 3
            end

            if error_center >= heave_length - 3
                error_center = heave_length - 3
            end
            
            # User-selected offset either side of GPS error
            lower_offset = upper_offset = 120

            if error_center <= lower_offset
                lower_offset = error_center - 1
            end

            if error_center + upper_offset > heave_length
                upper_offset = heave_length - error_center
            end

            # Ensure there are at least 3 points for fitting
            lower_offset = max(lower_offset, 2)
            upper_offset = max(upper_offset, 2)
    
            # Handle edge cases
            left_side_points = max(1, error_center - lower_offset):error_center
            right_side_points = error_center:min(heave_length, error_center + upper_offset)

            # Fit curve to subset of heave before GPS error
            fit1 = curve_fit(Polynomial, left_side_points, heave[left_side_points], 2)
            yfit1 = fit1.(left_side_points)
            yfit1[end] = 0.0  # set the last point of the left fit to 0

            # Fit curve to subset of heave after GPS error
            fit2 = curve_fit(Polynomial, right_side_points, heave[right_side_points], 2)
            yfit2 = fit2.(right_side_points)
            yfit2[1] = 0.0  # set the first point of the right fit to 0

            # Apply polynomial results to WSEs on both sides of GPS error
            heave[left_side_points] .= heave[left_side_points] - yfit1
            heave[right_side_points] .= heave[right_side_points] - yfit2
            heave[ii] = 0.0  # set WSE at GPS error location to 0

        end
    
    end

    return(heave)
    
end  # fix_gps_errors()


#######################################################################################################
#######################################################################################################
#######################################################################################################

ii = 1316

# Set time stamps
timestamps = RDT_df.Date[ii] .+ Microsecond.(collect(0:length(RDT_df.Heave[ii])-1) / 1.28 * 1000000)

p1 = plot(size=(1000,600))

p1 = plot!(timestamps, RDT_df.Heave[ii], lc=:blue, lw=:0.5, label="")

# Find indices of all values equal to 1
gps_flag = findall(x -> x == 1, RDT_df.GPS_flag[ii])

for ii in gps_flag
    p1 = vline!([timestamps[ii]], lw=1, c=:red, label="")
end
    
plot_p1 = plot(p1, 
    size = (1200, 400), title=Dates.format(RDT_df.Date[ii], "yyyy-mm-dd HH:MM")*" UTC",
    xlims=(timestamps[begin],timestamps[end]), #ylims=(-Inf,Inf),
    ylabel="WSE (m)",
    framestyle = :box, fg_legend=:transparent, legend=:bottomleft,
    leftmargin = 15Plots.mm, grid=true, gridlinewidth=0.5, gridcolor=:lightgrey, gridstyle=:dot, gridalpha=1)

display(plot_p1)

p1 = plot()

p1 = plot!(RDT_df.f2[ii], RDT_df.Pden2[ii], lc=:blue, lw=:2, label=RDT_df.Date[ii], )

plot_p1 = plot(p1, #yaxis=:log,
    size=(600,400), xlim=(0,0.64), ylim=(0,Inf), 
    xlabel="Frequency (Hz)", ylabel="S(f) (m²/Hz)",
    framestyle = :box, fg_legend=:transparent, legend=:topright,
    leftmargin = 20Plots.mm, grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)

display(plot_p1)

fixed_heave = fix_gps_errors(RDT_df.Heave[ii], RDT_df.Date[ii], RDT_df.GPS_flag[ii])

# Calculate spectra to each row in RDT_df.Heave using Welch's method
ps_w = welch_pgram(fixed_heave, 256, 128; onesided=true, nfft=256, fs=sample_frequency, window=hanning)
f2_fixed = freq(ps_w)
Pden2_fixed = power(ps_w)

using Plots
using Dates: Microsecond

# Set time stamps
timestamps = RDT_df.Date[ii] .+ Microsecond.(collect(0:length(RDT_df.Heave[ii])-1) / 1.28 * 1000000)

p1 = plot(size=(1000,600))

p1 = plot!(timestamps, RDT_df.Heave[ii], lc=:yellow, lw=3, label="")
p1 = plot!(timestamps, fixed_heave, lc=:blue, label="")

# Find indices of all values equal to 1
gps_flag = findall(x -> x == 1, RDT_df.GPS_flag[ii])

for ii in gps_flag
    p1 = vline!([timestamps[ii]], lw=1, c=:red, label="")
end
    
plot_p1 = plot(p1, 
    size = (1200, 400), title=Dates.format(RDT_df.Date[ii], "yyyy-mm-dd HH:MM")*" UTC",
    xlims=(timestamps[begin],timestamps[end]), #ylims=(-Inf,Inf),
    ylabel="WSE (m)",
    framestyle = :box, fg_legend=:transparent, legend=:bottomleft,
    leftmargin = 15Plots.mm, grid=true, gridlinewidth=0.5, gridcolor=:lightgrey, gridstyle=:dot, gridalpha=1)

display(plot_p1)

p1 = plot()

p1 = plot!(RDT_df.f2[ii], RDT_df.Pden2[ii], lc=:yellow, lw=:3, label=RDT_df.Date[ii], )
p1 = plot!(f2_fixed, Pden2_fixed, lc=:blue, lw=:1, label=RDT_df.Date[ii], )

plot_p1 = plot(p1, #yaxis=:log,
    size=(600,400), xlim=(0,0.64), ylim=(0,Inf), 
    xlabel="Frequency (Hz)", ylabel="S(f) (m²/Hz)",
    framestyle = :box, fg_legend=:transparent, legend=:topright,
    leftmargin = 20Plots.mm, grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)

display(plot_p1)

In [ ]:
using Tk: Toplevel, tcl, Frame, pack, Treeview, scrollbars_add, Button, bind, get_value
using Plots
using Dates: Dates, Microsecond
using CurveFit: curve_fit
    

gps_errors = findall(row -> row.GPS_errors > 0, eachrow(RDT_df))
gps_dates = RDT_df.Date[gps_errors]
bad_date_string_array = Dates.format.(gps_dates, "yyyy-mm-dd HH:MM")
good_date_string_array = Dates.format.(RDT_df.Date, "yyyy-mm-dd HH:MM")

w = Toplevel("Select Date", 235, 600)
tcl("pack", "propagate", w, false)
f = Frame(w)
pack(f, expand=true, fill="both")

f1 = Frame(f)
lb = Treeview(f1, date_string_array)
scrollbars_add(f1, lb)
pack(f1,  expand=true, fill="both")

tcl("ttk::style", "configure", "TButton", foreground="blue", font="arial 16 bold")
b = Button(f, "Ok")
pack(b)

bind(b, "command") do path
    
    # Select a record with GPS errors
    date_choice = get_value(lb);
    date_string = date_choice[1]
    ii = findfirst(x -> x == date_string, good_date_string_array)

    # Set time stamps
    timestamps = RDT_df.Date[ii] .+ Microsecond.(collect(0:length(RDT_df.Heave[ii])-1) / 1.28 * 1000000)
    
#==        
    p1 = plot(size=(1000,600))
    
    p1 = plot!(timestamps, RDT_df.Heave[ii], lc=:blue, lw=:0.5, label="")
    
    # Find indices of all values equal to 1
    gps_flag = findall(x -> x == 1, RDT_df.GPS_flag[ii])
    
    for ii in gps_flag
        p1 = vline!([timestamps[ii]], lw=1, c=:red, label="")
    end
    plot_p1 = plot(p1, 
        size = (1200, 400), title=Dates.format(RDT_df.Date[ii], "yyyy-mm-dd HH:MM")*" UTC",
        xlims=(timestamps[begin],timestamps[end]), #ylims=(-Inf,Inf),
        ylabel="WSE (m)",
        framestyle = :box, fg_legend=:transparent, legend=:bottomleft,
        leftmargin = 15Plots.mm, grid=true, gridlinewidth=0.5, gridcolor=:lightgrey, gridstyle=:dot, gridalpha=1)
    
    display(plot_p1)
    
    p1 = plot()
    
    p1 = plot!(RDT_df.f2[ii], RDT_df.Pden2[ii], lc=:blue, lw=:2, label=RDT_df.Date[ii], )
    
    plot_p1 = plot(p1, #yaxis=:log,
        size=(600,400), xlim=(0,0.64), ylim=(0,Inf), 
        xlabel="Frequency (Hz)", ylabel="S(f) (m²/Hz)",
        framestyle = :box, fg_legend=:transparent, legend=:topright,
        leftmargin = 20Plots.mm, grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)
    
    display(plot_p1)
==#    
    fixed_heave = fix_gps_errors(RDT_df.Heave[ii], RDT_df.Date[ii], RDT_df.GPS_flag[ii])
    
    # Calculate spectra to each row in RDT_df.Heave using Welch's method
    ps_w = welch_pgram(fixed_heave, 256, 128; onesided=true, nfft=256, fs=sample_frequency, window=hanning)
    f2_fixed = freq(ps_w)
    Pden2_fixed = power(ps_w)
    
    # Set time stamps
    timestamps = RDT_df.Date[ii] .+ Microsecond.(collect(0:length(RDT_df.Heave[ii])-1) / 1.28 * 1000000)
    
    p1 = plot(size=(1000,600))
    
    p1 = plot!(timestamps, RDT_df.Heave[ii], lc=:yellow, lw=3, label="")
    p1 = plot!(timestamps, fixed_heave, lc=:blue, label="")
    
    # Find indices of all values equal to 1
    gps_flag = findall(x -> x == 1, RDT_df.GPS_flag[ii])
    
    for ii in gps_flag
        p1 = vline!([timestamps[ii]], lw=1, c=:red, label="")
    end
        
    plot_p1 = plot(p1, 
        size = (1200, 400), title=Dates.format(RDT_df.Date[ii], "yyyy-mm-dd HH:MM")*" UTC",
        xlims=(timestamps[begin],timestamps[end]), #ylims=(-Inf,Inf),
        ylabel="WSE (m)",
        framestyle = :box, fg_legend=:transparent, legend=:bottomleft,
        leftmargin = 15Plots.mm, grid=true, gridlinewidth=0.5, gridcolor=:lightgrey, gridstyle=:dot, gridalpha=1)
    
    display(plot_p1)
    
    p1 = plot()
    
    p1 = plot!(RDT_df.f2[ii], RDT_df.Pden2[ii], lc=:yellow, lw=:3, label=RDT_df.Date[ii], )
    p1 = plot!(f2_fixed, Pden2_fixed, lc=:blue, lw=:1, label=RDT_df.Date[ii], )
    
    plot_p1 = plot(p1, #yaxis=:log,
        size=(600,400), xlim=(0,0.64), ylim=(0,Inf), 
        xlabel="Frequency (Hz)", ylabel="S(f) (m²/Hz)",
        framestyle = :box, fg_legend=:transparent, legend=:topright,
        leftmargin = 20Plots.mm, grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)
    
    display(plot_p1)

end

### Use Seralize to save df to .bin file

In [ ]:
using CodecZlib: GzipCompressorStream
using Serialization: serialize

# Serialize and compress the DataFrame to a file
outfil = ".\\Data\\" * split(directory_path,"\\")[end-1]*"_"*Dates.format(Date(Date(RDT_df.Date[1])), "yyyy-mm-dd")*
    "_to_"*Dates.format(Date(Date(RDT_df.Date[end])), "yyyy-mm-dd")*".bin"

println("Writing binary-formatted data to ",outfil)
flush(stdout)

@time begin

    open(outfil, "w") do io
        gz = GzipCompressorStream(io)    # Create a Gzip compressor stream
        serialize(gz, RDT_df)            # Serialize the DataFrame and write it to the compressed stream
        close(gz)                        # Close the compressor stream to ensure all data is written
    end

end

### Use Serialize to read gzipped .bin file to df

In [ ]:
using CodecZlib: GzipDecompressorStream
using Serialization: deserialize
using NativeFileDialog: pick_file
using DataFrames: DataFrame

##include("../Split_Spectra/Split_Spectra_Tools.jl")
##include("C://Users//PC1//Julia_programs//Split_Spectra//Split_Spectra_Tools.jl")

# Widen screen for better viewing
display(HTML("<style>:root { --jp-notebook-max-width: 80% !important; }</style>"))

function read_gzip_file(io)
###########################
    
    gz = GzipDecompressorStream(io)                # Create a Gzip decompressor stream
    deserialized_RDT_df = deserialize(gz) # Deserialize the DataFrame from the decompressed stream
    close(gz)                                      # Close the decompressor stream
    
    return(deserialized_RDT_df)
    
end    # read_gzip_file()


#######################################################################################################
#######################################################################################################
#######################################################################################################


# Select the binary file
# infil = pick_file("C:\\Users\\PC1\\Julia_programs\\Datawell\\Read_RDT\\Data\\", filterlist="*bin")
infil = pick_file("C:\\Users\\Jim\\Julia_programs\\Data")
    
println("Selected ", infil)

# declare the Heave, North, and West values from the Hex data
deserialized_RDT_df = DataFrame(
    Date = DateTime[], 
    Heave = Vector{Float64}[], 
    North = Vector{Float64}[], 
    West = Vector{Float64}[], 
    GPS_flag = Vector{Int16}[], 
    GPS_errors = Int64[], 
    f2 = Vector{Float64}[], 
    Pden2 = Vector{Float64}[], 
)

@time begin
    
    # Deserialize the DataFrame from the file
    deserialized_RDT_df = open(read_gzip_file, infil, "r")
    
end

println("Done!")

In [ ]:
deserialized_RDT_df

In [ ]:
RDT_df